<a href="https://colab.research.google.com/github/hyejeong14/BigdataAnalysisTest/blob/main/%EB%B9%85%EB%B6%84%EA%B8%B0_%EB%B6%84%EC%84%9D%EC%8B%A4%EC%8A%B5mtcars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#데이터 로드

In [1]:
!git clone https://github.com/AnalyticsKnight/yemoonsaBigdata

Cloning into 'yemoonsaBigdata'...
remote: Enumerating objects: 632, done.
remote: Counting objects: 100% (269/269), done.
remote: Compressing objects: 100% (163/163), done.
remote: Total 632 (delta 134), reused 180 (delta 106), pack-reused 363
Receiving objects: 100% (632/632), 10.75 MiB | 21.01 MiB/s, done.
Resolving deltas: 100% (319/319), done.


In [2]:
import pandas as pd
import numpy as np
import sklearn.preprocessing

In [3]:
X_test = pd.read_csv('/content/yemoonsaBigdata/datasets/Part2/mpg_X_test.csv')
X_train = pd.read_csv('/content/yemoonsaBigdata/datasets/Part2/mpg_X_train.csv')
y_train = pd.read_csv('/content/yemoonsaBigdata/datasets/Part2/mpg_y_train.csv')

#데이터 정보 확인

In [4]:
print(X_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278 entries, 0 to 277
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          278 non-null    object 
 1   mpg           278 non-null    float64
 2   cylinders     278 non-null    int64  
 3   displacement  278 non-null    float64
 4   horsepower    274 non-null    float64
 5   weight        278 non-null    int64  
 6   acceleration  278 non-null    float64
 7   model_year    278 non-null    int64  
dtypes: float64(4), int64(3), object(1)
memory usage: 17.5+ KB
None


#데이터 전처리

In [5]:
#결측치 처리 impute
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
X_train[['horsepower']]=imputer.fit_transform(X_train[['horsepower']])
X_test[['horsepower']]=imputer.fit_transform(X_test[['horsepower']])

In [6]:
print(X_train.describe())

              mpg   cylinders  displacement  horsepower       weight  \
count  278.000000  278.000000    278.000000  278.000000   278.000000   
mean    23.732734    5.374101    189.994604  103.383212  2948.464029   
std      7.647295    1.677084    105.471423   38.695458   862.949746   
min     10.000000    3.000000     68.000000   46.000000  1613.000000   
25%     18.000000    4.000000     98.000000   75.000000  2206.250000   
50%     23.000000    4.000000    140.500000   90.500000  2737.500000   
75%     29.000000    6.000000    258.000000  118.750000  3560.000000   
max     46.600000    8.000000    455.000000  230.000000  5140.000000   

       acceleration  model_year  
count    278.000000  278.000000  
mean      15.580216   76.057554  
std        2.745907    3.605591  
min        8.000000   70.000000  
25%       14.000000   73.000000  
50%       15.500000   76.000000  
75%       17.000000   79.000000  
max       24.800000   82.000000  


In [7]:
#변수의 형태별로 리스트로 저장 => 전처리 진행 시 편리함
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278 entries, 0 to 277
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          278 non-null    object 
 1   mpg           278 non-null    float64
 2   cylinders     278 non-null    int64  
 3   displacement  278 non-null    float64
 4   horsepower    278 non-null    float64
 5   weight        278 non-null    int64  
 6   acceleration  278 non-null    float64
 7   model_year    278 non-null    int64  
dtypes: float64(4), int64(3), object(1)
memory usage: 17.5+ KB


In [8]:
COL_DEL = ['name'] #분석에 영향x drop
COL_NUM = ['mpg','cylinders','displacement','horsepower','weight','acceleration','model_year']
COL_CAT = []  #카테고리(범주)형 => 인코딩 변환
COL_Y = ['isUSA']

X_train = X_train.iloc[:,1:]
X_test = X_test.iloc[:,1:]

#데이터 모형 구축

In [9]:
#데이터 분할
from sklearn.model_selection import train_test_split
X_tr,X_val,y_tr,y_val = train_test_split(X_train,y_train,test_size=0.3)

In [10]:
#수치형 변수 스케일링
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_tr = scaler.fit_transform(X_tr[COL_NUM])
X_val = scaler.transform(X_val[COL_NUM])
X_test = scaler.transform(X_test[COL_NUM])

In [11]:
#KNN 분류 모델
from sklearn.neighbors import KNeighborsClassifier

modelKNN = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
modelKNN.fit(X_tr,y_tr.values)

/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier(metric='euclidean')

In [12]:
y_tr

,isUSA
193,0
202,0
175,0
271,0
252,1
...,...
17,1
54,0
46,1
254,1


In [13]:
from sklearn.tree import DecisionTreeClassifier
modelDT = DecisionTreeClassifier(max_depth=10)
modelDT.fit(X_tr,y_tr)

DecisionTreeClassifier(max_depth=10)

In [14]:
y_val_pred_probaKNN = modelKNN.predict_proba(X_val)
y_val_pred_probaDT = modelDT.predict_proba(X_val)

In [16]:
y_val_proba = modelKNN.predict(X_val)

In [18]:
from sklearn.metrics import roc_auc_score

scoreKNN = roc_auc_score(y_val,y_val_pred_probaKNN[:,1])
scoreDT = roc_auc_score(y_val,y_val_pred_probaDT[:,1])

print(scoreKNN,scoreDT)

0.8259061200237672 0.82174688057041


In [26]:
best_model = None
best_score=0

for i in range(2,10):
  model = KNeighborsClassifier(n_neighbors=i,metric='euclidean')
  model.fit(X_tr,y_tr.values.ravel())
  y_val_pred_proba =model.predict_proba(X_val)
  score = roc_auc_score(y_val,y_val_pred_proba[:,1])
  print(i,"개의 이웃 확인 : ",score)

  if best_score <= score :
    best_score = score
    best_model = model

print(best_model)

2 개의 이웃 확인 :  0.8137254901960784
3 개의 이웃 확인 :  0.8199643493761141
4 개의 이웃 확인 :  0.8345216874628638
5 개의 이웃 확인 :  0.8259061200237672
6 개의 이웃 확인 :  0.8262032085561497
7 개의 이웃 확인 :  0.8458110516934048
8 개의 이웃 확인 :  0.8499702911467619
9 개의 이웃 확인 :  0.8300653594771242
KNeighborsClassifier(metric='euclidean', n_neighbors=8)


In [27]:
print(best_model.predict_proba(X_test))

[[0.875 0.125]
 [0.25  0.75 ]
 [0.    1.   ]
 [0.    1.   ]
 [0.875 0.125]
 [0.875 0.125]
 [0.625 0.375]
 [0.75  0.25 ]
 [0.625 0.375]
 [0.5   0.5  ]
 [0.    1.   ]
 [0.25  0.75 ]
 [0.    1.   ]
 [0.5   0.5  ]
 [0.75  0.25 ]
 [0.875 0.125]
 [0.    1.   ]
 [0.    1.   ]
 [0.875 0.125]
 [0.    1.   ]
 [0.    1.   ]
 [0.    1.   ]
 [0.    1.   ]
 [0.    1.   ]
 [0.    1.   ]
 [0.625 0.375]
 [0.875 0.125]
 [0.    1.   ]
 [0.    1.   ]
 [0.375 0.625]
 [1.    0.   ]
 [0.75  0.25 ]
 [0.625 0.375]
 [0.125 0.875]
 [0.    1.   ]
 [0.375 0.625]
 [0.5   0.5  ]
 [0.    1.   ]
 [0.875 0.125]
 [0.25  0.75 ]
 [0.    1.   ]
 [0.875 0.125]
 [1.    0.   ]
 [0.5   0.5  ]
 [0.875 0.125]
 [0.625 0.375]
 [0.625 0.375]
 [0.    1.   ]
 [0.875 0.125]
 [0.    1.   ]
 [1.    0.   ]
 [0.875 0.125]
 [0.    1.   ]
 [0.875 0.125]
 [0.    1.   ]
 [0.    1.   ]
 [0.    1.   ]
 [0.25  0.75 ]
 [0.    1.   ]
 [0.625 0.375]
 [0.    1.   ]
 [0.25  0.75 ]
 [0.    1.   ]
 [0.    1.   ]
 [0.125 0.875]
 [0.    1.   ]
 [0.    1.

In [28]:
pred = best_model.predict_proba(X_test)[:,1]
print(pred)

[0.125 0.75  1.    1.    0.125 0.125 0.375 0.25  0.375 0.5   1.    0.75
 1.    0.5   0.25  0.125 1.    1.    0.125 1.    1.    1.    1.    1.
 1.    0.375 0.125 1.    1.    0.625 0.    0.25  0.375 0.875 1.    0.625
 0.5   1.    0.125 0.75  1.    0.125 0.    0.5   0.125 0.375 0.375 1.
 0.125 1.    0.    0.125 1.    0.125 1.    1.    1.    0.75  1.    0.375
 1.    0.75  1.    1.    0.875 1.    1.    0.125 0.875 0.125 0.875 1.
 0.625 0.    1.    0.25  1.    0.25  0.25  1.    0.875 1.    0.    1.
 1.    0.875 0.125 0.125 0.125 1.    1.    1.    1.    1.    1.    0.875
 0.125 1.    1.    1.    1.    1.    1.    1.    1.    0.125 0.375 0.625
 0.125 1.    0.125 0.875 0.125 1.    1.    0.    1.    1.    1.    0.   ]


In [31]:
pd.DataFrame({'isUSA':pred}).to_csv('./003000000.csv',index=False)